### Import modules


In [1]:
import kagglehub, chess, chess.pgn, chess.svg
import pandas as pd, numpy as np, io, os, math, random, time, sys
from collections import deque
from dataclasses import dataclass
from tqdm import tqdm
from multiprocessing import Pool, cpu_count, freeze_support
from typing import Optional, List, Tuple
from IPython.display import display, clear_output, SVG, HTML

### Load and parse data


In [2]:
def load_and_parse_csv(file_path, max_games=5000):
    """
    Load CSV with PGN entries, filter standard chess games,
    produce DataFrame of positions: ['FEN','Result'].
    """
    try:
        df = pd.read_csv(file_path, usecols=['pgn', 'white_result', 'black_result', 'rules'])
    except Exception as e:
        print(f"Error loading CSV: {e}")
        return None

    df = df[df['rules'] == 'chess'].reset_index(drop=True)
    data, game_count = [], 0

    # Define sets of keywords for result classification
    win_terms = {"win"}
    lose_terms = {"checkmated", "resigned", "timeout"}
    draw_terms = {"stalemate", "repetition", "insufficient", "draw"}

    # Use tqdm as a context manager
    with tqdm(total=max_games, desc="Parsing games", ncols=80) as pbar:
        # Iterate through each row (game) in the DataFrame
        for _, row in df.iterrows():
            if game_count >= max_games:
                break  # Stop iterating rows once we hit our target

            # Extract result terms
            w, b = row['white_result'].lower(), row['black_result'].lower()
            # Assign numeric outcomes: +1 = White win, -1 = Black win, 0 = Draw
            if w in win_terms and b in lose_terms:
                outcome = 1.0
            elif b in win_terms and w in lose_terms:
                outcome = -1.0
            elif w in draw_terms or b in draw_terms:
                outcome = 0.0
            else:
                continue  # Skip invalid/unrecognized results

            # Start from the initial board position
            try:
                # Convert PGN string to a game object
                game = chess.pgn.read_game(io.StringIO(str(row['pgn'])))
                if not game:
                    continue
                
                # For every move in the mainline of the game:
                board = game.board()
                for move in game.mainline_moves():
                    # Store FEN representation and game result for each position
                    data.append({"FEN": board.fen(), "Result": outcome})
                    board.push(move)  # Apply the move to get next position
            except Exception:
                continue

            # Only increment count and update bar if game was valid
            game_count += 1
            pbar.update(1) # Manually increment the progress bar

            # Log progress to the tqdm bar description itself
            if game_count % 100 == 0:
                 pbar.set_description(f"Parsed {game_count}/{max_games}")

    print(f"Parsed {len(data)} positions from {game_count} valid games.")
    # Return DataFrame of all positions with outcomes
    print("\nExtracting features......")
    return pd.DataFrame(data) if data else None

### Features Extraction from data


In [3]:
class ChessFeatureExtractor:
    """
    Convert chess.Board -> numeric 5-dim feature vector:
    [material, mobility, center, king_safety, pawn_structure]
    """
    def __init__(self):
        # Central squares important for positional control
        self.center_squares = [chess.D4, chess.D5, chess.E4, chess.E5]
        # Simplified piece value mapping (pawn=1, knight=3, bishop=3, rook=5, queen=9, king=0)
        self.piece_values = {1: 1, 2: 3, 3: 3, 4: 5, 5: 9, 6: 0}

    def extract_features(self, board):
        """Compute 5 handcrafted numerical features from a chess board."""
        material = self._material_balance(board)
        mobility = len(list(board.legal_moves))  # Count legal moves
        center = self._center_control(board)
        king_safety = self._king_safety(board)
        pawn_structure = self._pawn_structure(board)

        # Combine features into a fixed-length numpy array
        return np.array([
            material / 39,         # material difference normalized
            mobility / 60,         # legal moves range (rough max)
            center / 4,            # 4 central squares
            king_safety / 8,       # safety score normalization
            pawn_structure / 8     # structure penalty normalization
        ], dtype=float)

    def fen_to_features(self, fen):
        """Wrapper: Convert FEN string into feature vector."""
        board = chess.Board(fen)
        return self.extract_features(board)

    def _material_balance(self, board):
        """Compute net material: (white value - black value)."""
        total = 0
        for piece in board.piece_map().values():
            value = self.piece_values[piece.piece_type]
            total += value if piece.color == chess.WHITE else -value
        return total

    def _center_control(self, board):
        """Count white - black pieces occupying central squares."""
        control = 0
        for sq in self.center_squares:
            piece = board.piece_at(sq)
            if piece:
                control += 1 if piece.color == chess.WHITE else -1
        return control

    def _king_safety(self, board):
        """Roughly estimate king safety by counting nearby enemy attacks."""
        score = 0
        try:
            # White king under attack by black pieces
            for sq in board.attacks(board.king(chess.WHITE)):
                if board.piece_at(sq) and board.piece_at(sq).color == chess.BLACK:
                    score -= 1
            # Black king under attack by white pieces
            for sq in board.attacks(board.king(chess.BLACK)):
                if board.piece_at(sq) and board.piece_at(sq).color == chess.WHITE:
                    score += 1
        except:
            pass
        return score

    def _pawn_structure(self, board):
        """Apply penalties for doubled or isolated pawns."""
        penalty = 0
        for color in [chess.WHITE, chess.BLACK]:
            pawns = board.pieces(chess.PAWN, color)
            files = [chess.square_file(p) for p in pawns]

            if not files:
                continue

            # Doubled pawns: multiple pawns on the same file
            doubled = [f for f in set(files) if files.count(f) > 1]
            penalty += (-1 if color == chess.WHITE else 1) * len(doubled)

            # Isolated pawns: no friendly pawns on adjacent files
            isolated = [f for f in files if (f - 1 not in files and f + 1 not in files)]
            penalty += (-0.5 if color == chess.WHITE else 0.5) * len(isolated)
        return penalty

In [4]:
# ===========================================================================
#  Define Game State
# ===========================================================================
@dataclass
class GameState:
    fen: str
    features: np.ndarray
    predicted_value: float
    actual_outcome: Optional[float] = None

# ===========================================================================
#  Use of MiniMax to optimize moves
# ===========================================================================
def order_moves(board, aggression=0.5):
    moves = list(board.legal_moves)
    # Weight captures and checks dynamically based on opponent aggression
    moves.sort(key=lambda m: (
        board.is_capture(m)*(10 + aggression*5) + 
        board.gives_check(m)*(5 + aggression*3)
    ), reverse=True)
    return moves

class MinimaxSearcher:
    def __init__(self, evaluator, depth=3, opponent_model=None):
        self.evaluator = evaluator
        self.depth = depth
        self.opponent_model = opponent_model

    def minimax(self, board, depth, alpha=-math.inf, beta=math.inf, maximizing=True):
        if depth==0 or board.is_game_over():
            return self.evaluator.evaluate_board(board, depth)
        aggr = 0.5 if self.opponent_model is None else self.opponent_model.predict_aggressiveness()
        moves = order_moves(board, aggr)
        if maximizing:
            # Maximizing player (White)
            max_eval = -math.inf
            for move in moves:
                board.push(move)
                eval = self.minimax(board, depth-1, alpha, beta, False)
                board.pop()
                max_eval = max(max_eval, eval)
                alpha = max(alpha, eval)
                if beta <= alpha:
                    break  # Prune subtree
            return max_eval
        else:
            # Minimizing player (Black)
            min_eval = math.inf
            for move in moves:
                board.push(move)
                eval = self.minimax(board, depth-1, alpha, beta, True)
                board.pop()
                min_eval = min(min_eval, eval)
                beta = min(beta, eval)
                if beta <= alpha:
                    break  # Prune subtree
            return min_eval

    def best_move(self, board):
        """Returns best move at root node based on minimax evaluation."""
        aggr = 0.5 if self.opponent_model is None else self.opponent_model.predict_aggressiveness()
        best_val = -math.inf
        best_move = None

        for move in order_moves(board, aggr):
            board.push(move)
            move_val = self.minimax(board, self.depth - 1, maximizing=False)
            board.pop()

            if move_val > best_val:
                best_val = move_val
                best_move = move

        return best_move

# ===========================================================================  
#  Parallel processing for Minimax  
# ===========================================================================  
class ParallelMinimaxSearcher(MinimaxSearcher):
    """Parallelized Minimax using multiprocessing for top-level move evaluation."""
    def __init__(self, evaluator, depth=3, opponent_model=None):
        super().__init__(evaluator, depth, opponent_model)
        self.num_workers = max(1, cpu_count() - 1)
        print(f"ParallelSearcher initialized with depth={depth}, workers={self.num_workers}")

    def evaluate_move_parallel(self, move_board_tuple):
        move, board_after_move = move_board_tuple
        score = self.minimax(board_after_move, self.depth - 1, maximizing=False)
        return (move, score)

    def best_move(self, board):
        """Evaluate all root moves in parallel."""
        moves = order_moves(board)
        if not moves:
            return None

        boards_after_move = []
        for m in moves:
            b_copy = board.copy()
            b_copy.push(m)
            boards_after_move.append((m, b_copy))

        try:
            from tqdm import tqdm
            with Pool(processes=self.num_workers) as pool:
                results = list(tqdm(pool.imap(self.evaluate_move_parallel, boards_after_move),
                                    total=len(boards_after_move),
                                    desc="Parallel search", leave=False, ncols=80))
            best_move, best_score = max(results, key=lambda item: item[1])
            return best_move
        except Exception as e:
            print(f"⚠️ Parallel search error: {e}, falling back to serial.")
            return super().best_move(board)

### Conjugate Gradient Solver


In [5]:
def Conjugate_Solve(A_mul, b, x0=None, tol=1e-6, maxiter=1000):
    """
    Solves A x = b approximately using the Conjugate Gradient (CG) method.

    Parameters
    ----------
    A_mul : callable
        Function that computes A @ v (matrix-vector product).
    b : ndarray
        Right-hand side vector.
    x0 : ndarray, optional
        Initial guess for the solution.
    tol : float
        Tolerance for convergence.
    maxiter : int
        Maximum iterations.

    Returns
    -------
    x : ndarray
        Approximate solution vector.
    """
    n = len(b)
    x = np.zeros_like(b) if x0 is None else x0.copy()
    r = b - A_mul(x)
    p = r.copy()
    rsold = np.dot(r, r)
    if rsold == 0:
        return x

    for i in range(min(maxiter, n * 10)):
        Ap = A_mul(p)
        denom = np.dot(p, Ap) + 1e-16
        alpha = rsold / denom
        if np.isnan(alpha) or np.isinf(alpha):
            break
        x += alpha * p
        r -= alpha * Ap
        rsnew = np.dot(r, r)
        if math.sqrt(rsnew) < tol:
            break
        p = r + (rsnew / (rsold + 1e-16)) * p
        rsold = rsnew

    return x

### Golden Section Search


In [6]:
def Golden_Section_Search(objective_fn, a=0.4, b=2.5, iters=50):
    """
    Standard golden section search to minimize a scalar function.

    Parameters
    ----------
    objective_fn : callable
        Function f(x) to minimize.
    a, b : float
        Interval [a, b] to search.
    iters : int
        Number of iterations.

    Returns
    -------
    x_opt : float
        Approximate minimizer of f(x).
    """
    phi = (math.sqrt(5) - 1) / 2.0
    c = b - phi * (b - a)
    d = a + phi * (b - a)
    fc = objective_fn(c)
    fd = objective_fn(d)

    for _ in range(iters):
        if fc < fd:
            b, d, fd = d, c, fc
            c = b - phi * (b - a)
            fc = objective_fn(c)
        else:
            a, c, fc = c, d, fd
            d = a + phi * (b - a)
            fd = objective_fn(d)

    return (a + b) / 2.0

### Dynamic Chess Evaluator (using CG & Golden Section)


In [7]:
class DynamicChessEvaluator:
    """
    Online adaptive evaluator:
     - W: feature weights (updated by solving normal equations via CG)
     - x: aggressiveness scalar (optimized by Golden Section Search)
    Replay buffer stores GameState entries; updates are triggered after games.
    """

    def __init__(self, extractor: ChessFeatureExtractor, initial_x: float = 1.0, lambda_reg: float = 0.01):
        self.extractor = extractor
        self.W = np.random.randn(5) * 0.01
        self.x = float(initial_x)
        self.lambda_reg = float(lambda_reg)
        self.replay_buffer = deque(maxlen=2000)
        self.position_count = 0

    def evaluate_board(self, board: chess.Board, depth: int = 0) -> float:
        # Terminal handling
        if board.is_checkmate():
            return (-1e6 - depth) if board.turn else (1e6 + depth)
        if board.is_stalemate() or board.is_insufficient_material() or board.can_claim_draw():
            return 0.0

        f = self.extractor.extract_features(board)
        x_dyn = self._adaptive_x(f[0])
        return float(np.dot(self.W, f * x_dyn))

    def _adaptive_x(self, material_balance):
        # heuristic: scale x based on material imbalance
        if material_balance < -3:
            return min(self.x * 1.5, 3.0)
        elif material_balance > 3:
            return max(self.x * 0.6, 0.2)
        return np.clip(self.x * (1 + 0.2 * np.tanh(material_balance / 3)), 0.2, 3.0)

    def record_experience(self, state: GameState):
        self.replay_buffer.append(state)
        self.position_count += 1

    def conjugate_gradient_update_W(self, batch_size=500):
        """
        Fit W by minimizing mean squared error via Conjugate Gradient.
        Solves: ((2/n)*XᵀX + λI)W = (2/n)*Xᵀy
        """
        recent = [s for s in list(self.replay_buffer) if s.actual_outcome is not None]
        if len(recent) < 10:
            return False

        recent = recent[-batch_size:]
        X = np.vstack([s.features for s in recent])
        y = np.array([s.actual_outcome for s in recent])
        n = len(y)
        Xs = X * self.x

        A_mat = (2.0 / n) * (Xs.T @ Xs) + self.lambda_reg * np.eye(Xs.shape[1])
        b_vec = (2.0 / n) * (Xs.T @ y)

        w_init = self.W.copy()
        self.W = Conjugate_Solve(lambda v: A_mat @ v, b_vec, x0=w_init, tol=1e-8, maxiter=1000)
        return True
    
    def incremental_update(self, features, outcome, lr=0.001):
        pred = np.dot(self.W, features * self.x)
        error = np.clip(outcome - pred, -10, 10)   # prevent blow-up
        grad = -2 * error * (features * self.x)
        self.W -= lr * grad
        self.x = np.clip(self.x + 0.001 * np.sign(error) * np.mean(features), 0.2, 3.0)

    def golden_section_optimize_x(self, sample_size=500, iters=25):
        """
        Uses Golden Section Search to find x that maximizes correlation
        between predicted and actual outcomes.
        """
        recent = [s for s in list(self.replay_buffer) if s.actual_outcome is not None]
        if len(recent) < 10:
            return False
        recent = recent[-sample_size:]

        def objective_fn(xv):
            preds = np.array([np.dot(self.W, s.features * xv) for s in recent])
            outs = np.array([s.actual_outcome for s in recent])
            corr = np.corrcoef(preds, outs)[0, 1]
            return -corr if not np.isnan(corr) else 0.0
        self.x = float(Golden_Section_Search(objective_fn, a=0.4, b=2.5, iters=iters))
        return True

### Chess AI


In [8]:
class OpponentModel:
    """Tracks opponent tendencies and predicts next move type."""
    def __init__(self):
        self.aggression_score = 0.5
        self.move_count = 0
        self.capture_ratio = 0.0
        self.check_ratio = 0.0

    def update(self, board, move):
        """Update aggression and play statistics after each opponent move."""
        self.move_count += 1
        if board.is_capture(move):
            self.aggression_score = min(1.0, self.aggression_score + 0.05)
            self.capture_ratio += 1
        elif board.gives_check(move):
            self.aggression_score = min(1.0, self.aggression_score + 0.02)
            self.check_ratio += 1
        else:
            self.aggression_score = max(0.0, self.aggression_score - 0.01)

    def predict_aggressiveness(self):
        """Estimate aggressiveness for predictive bias."""
        return self.aggression_score

class DynamicChessAI:
    """
    Controller that ties extractor, evaluator, and searcher.
    Usage:
        ai = DynamicChessAI(depth=3)
        move, score = ai.make_move(board)
        ai.update_outcome(result)  # after game finishes
    """
    def __init__(self, depth=3):
        self.extractor = ChessFeatureExtractor()
        self.evaluator = DynamicChessEvaluator(self.extractor)
        self.opponent_model = OpponentModel()
        self.searcher = ParallelMinimaxSearcher(self.evaluator, depth, opponent_model=self.opponent_model)
        self.move_history: List[GameState] = []

    def process_opponent_move(self, mv: chess.Move, board: chess.Board):
        self.opponent_model.update(board, mv)

    def make_move(self, board: chess.Board):
        features = self.extractor.extract_features(board)
        predicted = self.evaluator.evaluate_board(board)
        mv = self.searcher.best_move(board)
        # compute immediate target by evaluating position after mv
        if mv is not None:
            bcopy = board.copy()
            bcopy.push(mv)
            # serial shallow search for target (depth-1)
            target_val = self.searcher.minimax(bcopy, max(1, self.searcher.depth-1), maximizing=False)
            # one-step online update
            self.evaluator.incremental_update(features, target_val, lr=0.005)

        score = self.evaluator.evaluate_board(board)
        st = GameState(board.fen(), features, predicted)
        self.evaluator.record_experience(st)
        self.move_history.append(st)
        return mv, score

    def update_outcome(self, result: float):
        """
        When game ends, attach final outcome to each GameState in the game history,
        then feed them again to replay buffer (they already were there; we add labels)
        and trigger parameter updates.
        """
        for s in self.move_history:
            s.actual_outcome = result
            self.evaluator.record_experience(s)
        self.move_history.clear()
        # Update parameters after receiving labeled data
        updated_w = self.evaluator.conjugate_gradient_update_W()
        updated_x = self.evaluator.golden_section_optimize_x()
        return updated_w, updated_x

# ===========================================================================
#  Player and Opponent Classes
# ===========================================================================
class PlayerSimulator:
    """
    Simulates a baseline 'player'.
    Plays decently — prefers captures and checks, else random.
    Used as the 'You' or baseline agent.
    """
    def __init__(self, color=chess.WHITE):
        self.color = color

    def select_move(self, board: chess.Board):
        moves = list(board.legal_moves)
        if not moves:
            return None
        # Prefer captures
        capture_moves = [m for m in moves if board.is_capture(m)]
        if capture_moves:
            return random.choice(capture_moves)
        # Next preference: checks
        check_moves = [m for m in moves if board.gives_check(m)]
        if check_moves:
            return random.choice(check_moves)
        # Otherwise random legal move
        return random.choice(moves)

class ToughOpponent:
    """
    A stronger opponent that evaluates 1-ply ahead using positional heuristics.
    Factors in:
      - Material balance
      - Mobility
      - Center control
      - King safety
    This plays roughly like a classical engine with shallow lookahead.
    """
    def __init__(self, color=chess.BLACK):
        self.color = color
        self.extractor = ChessFeatureExtractor()

    def evaluate_move(self, board, move):
        # Simulate the move
        board.push(move)
        features = self.extractor.extract_features(board)
        score = (
            2.0 * features[0] +     # material
            0.6 * features[1] +     # mobility
            0.4 * features[2] -     # center control
            0.3 * abs(features[3])  # king safety penalty
        )
        board.pop()
        return score

    def select_move(self, board: chess.Board):
        moves = list(board.legal_moves)
        if not moves:
            return None
        # Evaluate all possible moves
        move_scores = [(m, self.evaluate_move(board, m)) for m in moves]
        # Choose move that maximizes position evaluation
        best_move, _ = max(move_scores, key=lambda x: x[1])
        return best_move

### Live Gameplay


In [9]:
# ===========================================================================
# ENVIRONMENT DETECTION
# ===========================================================================
def detect_env():
    if 'google.colab' in sys.modules:
        return 'colab'
    elif 'VSCODE_PID' in os.environ or 'vscode' in str(sys.argv).lower():
        return 'vscode'
    elif 'ipykernel' in sys.modules:
        return 'jupyter'
    else:
        return 'unknown'

ENV = detect_env()
print(f"Running in: {ENV}")

# ===========================================================================
# DYNAMIC BOARD DISPLAY
# ===========================================================================
def show_board(board, move=None, delay=0.7, size=300):
    """
    Display a chess board across VSCode, Jupyter, and Colab.
    Automatically handles rendering + move display + delay.
    
    Parameters:
        board : chess.Board object
        move  : chess.Move (optional) - the move just played
        delay : float - time delay between frames (seconds)
        size  : int - board size (pixels)
    """
    clear_output(wait=True)
    move_text = f"<b>Move:</b> {move.uci()}" if move else ""
    svg_data = chess.svg.board(board, size=size)

    if ENV in ['jupyter', 'colab']:
        display(HTML(f"{move_text}<br>"))
        display(SVG(svg_data))
    elif ENV == 'vscode':
        display(HTML(f"<div style='font-family:monospace; margin-bottom:4px'>{move_text}</div><div>{svg_data}</div>"))
    else:
        print(move_text)
        print(board)
    # Delay to visualize progression
    time.sleep(delay)

# ===========================================================================
# GAME PLAY  
# ===========================================================================
def play(player_type, n_games=5, opponent_type=None, depth=None, show_play=False, delay=0.5, csv_name=None):
    """
    Unified function for self-play or visualized play between the AI and chosen opponent.
    Now continuously updates CSV after each game and returns the full DataFrame.
    """
    VISUAL = (ENV in ['vscode', 'jupyter', 'colab'])
    is_dynamic_ai = isinstance(player_type, DynamicChessAI)

    # Setup searcher
    if is_dynamic_ai:
        if show_play:
            player_type.searcher = MinimaxSearcher(player_type.evaluator, depth, opponent_model=player_type.opponent_model)
            print(f"👀 Running in visual (serial) Minimax mode (depth={depth}).\n")
        else:
            player_type.searcher = ParallelMinimaxSearcher(player_type.evaluator, depth, opponent_model=player_type.opponent_model)
            print(f"🧠 Running in parallel Minimax mode (depth={depth}).\n")
    else:
        print(f"👀 Running simulation with simple '{type(player_type).__name__}'.\n")

    results, total_moves = [], []

    if opponent_type == "tough":
        opponent = ToughOpponent()
    elif opponent_type == "player":
        opponent = PlayerSimulator()
    else:
        raise ValueError(f"Unknown opponent_type: {opponent_type}")

    print(f"\n🎮 Starting {'visualized ' if show_play else ''}play: {n_games} games vs {opponent_type.upper()} opponent.\n")

    for g in tqdm(range(n_games), desc="Games", ncols=80):
        board = chess.Board()
        moves = 0

        while not board.is_game_over() and moves < 120:
            if board.turn == chess.WHITE:
                if isinstance(player_type, DynamicChessAI):
                    mv, sc = player_type.make_move(board)
                elif isinstance(player_type, PlayerSimulator):
                    mv = player_type.select_move(board)
                else:
                    raise ValueError("Unsupported AI type for White.")
            else:
                mv = opponent.select_move(board)
                if hasattr(player_type, "opponent_model"):
                    player_type.opponent_model.update(board, mv)

            if mv is None:
                break
            board.push(mv)
            moves += 1

            if show_play and VISUAL:
                show_board(board, move=mv, delay=delay)

        res_str = board.result()
        final = 1.0 if res_str == "1-0" else -1.0 if res_str == "0-1" else 0.0

        if is_dynamic_ai:
            player_type.update_outcome(final)

        results.append(final)
        total_moves.append(moves)

        # Create incremental DataFrame
        df_partial = pd.DataFrame({
            "Game_ID": range(1, len(results) + 1),
            "Result": results,
            "Moves": total_moves
        })

        # Write live progress to CSV
        if csv_name:
            df_partial.to_csv(csv_name, index=False)

        print(f"\nGame {g+1}/{n_games}: Result={res_str} | Moves={moves}")
        time.sleep(0.2)

    win_rate = sum(1 for r in results if r == 1.0) / len(results)
    print("\n✅ Play complete.")
    print(f"Win rate (as White): {win_rate:.2%}")

    if csv_name:
        print(f"💾 Final results saved to '{csv_name}'")

    return df_partial

Running in: vscode


In [10]:
# ===========================================================================
# Download and load the dataset
# ===========================================================================
path = kagglehub.dataset_download("adityajha1504/chesscom-user-games-60000-games")
file_path = os.path.join(path, 'club_games_data.csv')

df = load_and_parse_csv(file_path, max_games=5000)  # Load and parse the data into FEN positions + results
# Check for success
if df is None or df.empty:
   raise ValueError("Failed to load or parse the dataset properly.")
print(f"✅ Loaded {len(df)} positions from {file_path}")

# ===========================================================================
# Convert FEN strings to feature vectors
# ===========================================================================
extractor = ChessFeatureExtractor()
# Convert each FEN to its corresponding feature vector
X = np.array([extractor.fen_to_features(fen) for fen in df.FEN.values])
y = df.Result.values
print(f"✅ Feature matrix shape: {X.shape}")
print(f"✅ Target vector shape: {y.shape}")

Parsed 5000/5000: 100%|████████████████████| 5000/5000 [00:14<00:00, 346.09it/s]


Parsed 307385 positions from 5000 valid games.

Extracting features......
✅ Loaded 307385 positions from C:\Users\majum\.cache\kagglehub\datasets\adityajha1504\chesscom-user-games-60000-games\versions\1\club_games_data.csv
✅ Feature matrix shape: (307385, 5)
✅ Target vector shape: (307385,)


In [11]:
if __name__ == "__main__":
    # ===========================================================================
    # Initialize AI and bootstrap replay buffer with labeled examples
    # ===========================================================================
    player_sim = PlayerSimulator()
    dynamic_ai = DynamicChessAI(depth=3)
    
    n_boot = min(1200, len(X))
    idx = np.random.choice(len(X), size=n_boot, replace=False)
    for i in idx:
        # predicted value computed with current (random) W and x
        pred = float(np.dot(dynamic_ai.evaluator.W, X[i] * dynamic_ai.evaluator.x))
        st = GameState(df.FEN.values[i], X[i], pred, actual_outcome=float(y[i]))
        dynamic_ai.evaluator.record_experience(st)
    print(f"Bootstrapped replay buffer with {len(dynamic_ai.evaluator.replay_buffer)} labeled examples.\n")
    
    # --- Run Your Simulations ---
    
    # Simple player vs tough opponent
    df1 = play(player_sim, n_games=100, opponent_type="tough", depth=3,
            show_play=False, delay=0.1, csv_name="PlayerVsOpponent.csv")

    # Dynamic AI vs tough opponent
    df2 = play(dynamic_ai, n_games=100, opponent_type="tough", depth=3,
            show_play=True, delay=0.1, csv_name="AiVsOpponent.csv")

    # print("\n--- 3. Dynamic AI vs. Easier Opponent (PlayerSim) ---")
    # gameplay3 = play(dynamic_ai, n_games=10, opponent_type="player", depth=3, show_play=True, delay=0.5)

Games: 100%|████████████████████████████████| 100/100 [1:27:04<00:00, 16.98s/it]


Game 100/100: Result=1-0 | Moves=9


Games: 100%|████████████████████████████████| 100/100 [1:27:04<00:00, 52.25s/it]


✅ Play complete.
Win rate (as White): 57.00%
💾 Final results saved to 'AiVsOpponent.csv'
